In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "8"
DEVICE = 'cuda:0'


In [2]:
from tqdm.auto import tqdm
from typing import List
import re
import torch

/mount/arbeitsdaten/asr-2/vaethdk/virtualenvs/cts_en/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
torch.cuda.device_count()

1

In [8]:
import sys
sys.path.append('../../..')
print(os.path.realpath("."))

/mount/arbeitsdaten41/projekte/asr-2/vaethdk/cts_newcodebase_rollback/conversational-tree-search/generation/reimburse/llama


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

model_name_or_path = "TheBloke/upstage-llama-30b-instruct-2048-GPTQ"
model_basename = "gptq_model-4bit--1g"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          use_fast=True,
                                          cache_dir="/mount/arbeitsdaten/asr-2/vaethdk/resources/weights/",)

model = AutoGPTQForCausalLM.from_quantized("/mount/arbeitsdaten/asr-2/vaethdk/resources/weights/TheBloke--upstage-llama-30b-instruct-2048-GPTQ",
        # model_basename=model_basename,
        # revision="gptq-4bit-32g-actorder_True",
        use_safetensors=True,
        trust_remote_code=False,
        device="cuda:0",
        use_triton=use_triton,
        quantize_config=None)

CUDA extension not installed.
The safetensors archive passed at /mount/arbeitsdaten/asr-2/vaethdk/resources/weights/TheBloke--upstage-llama-30b-instruct-2048-GPTQ/gptq_model-4bit-32g.safetensors does not contain metadata. Make sure to save your model with the `save_pretrained` method. Defaulting to 'pt' metadata.
skip module injection for FusedLlamaMLPForQuantizedModel not support integrate without triton yet.


In [28]:
def generate_prompt(system: str, user: str) -> str:
    return f"""
    ### System:
    {system}

    ### User:
    {user}

    ### Assistant:"""

def generate_output(prompt: str, temperature: float = 0.7, max_new_tokens: int = 512) -> torch.FloatTensor:
    input_ids = tokenizer(prompt, return_tensors='pt').input_ids.cuda()
    output = model.generate(inputs=input_ids, temperature=temperature, max_new_tokens=max_new_tokens)
    return tokenizer.decode(output[0])

def parse_output(prompt: str, output: str) -> List[str]:
    # remove prompt from output first (ends at ### ASSISTANT: )
    return output.replace("<s>", "").strip()[len(prompt):].replace("</s>", "").strip()

In [19]:
from data.dataset import ReimburseGraphDataset, DataAugmentationLevel

data_generated_v1 = ReimburseGraphDataset('en/reimburse/train_graph.json', 'en/reimburse/train_answers.json', True, augmentation=DataAugmentationLevel.ARTIFICIAL_ONLY, augmentation_path="en/reimburse/generated/train_questions_v1.json", resource_dir="../../../resources")
data_generated_v1_ling = ReimburseGraphDataset('en/reimburse/train_graph.json', 'en/reimburse/train_answers.json', True, augmentation=DataAugmentationLevel.ARTIFICIAL_ONLY, augmentation_path="en/reimburse/generated/train_questions_v1_ling.json", resource_dir="../../../resources")

data_generated_v2 = ReimburseGraphDataset('en/reimburse/train_graph.json', 'en/reimburse/train_answers.json', True, augmentation=DataAugmentationLevel.ARTIFICIAL_ONLY, augmentation_path="en/reimburse/generated/train_questions_v2.json", resource_dir="../../../resources")
data_generated_v2_ling = ReimburseGraphDataset('en/reimburse/train_graph.json', 'en/reimburse/train_answers.json', True, augmentation=DataAugmentationLevel.ARTIFICIAL_ONLY, augmentation_path="en/reimburse/generated/train_questions_v2_ling.json", resource_dir="../../../resources")

data_generated_v3 = ReimburseGraphDataset('en/reimburse/train_graph.json', 'en/reimburse/train_answers.json', True, augmentation=DataAugmentationLevel.ARTIFICIAL_ONLY, augmentation_path="en/reimburse/generated/train_questions_v3.json", resource_dir="../../../resources")
data_generated_v3_ling = ReimburseGraphDataset('en/reimburse/train_graph.json', 'en/reimburse/train_answers.json', True, augmentation=DataAugmentationLevel.ARTIFICIAL_ONLY, augmentation_path="en/reimburse/generated/train_questions_v3_ling.json", resource_dir="../../../resources")

- Loading questions from  ../../../resources/en/reimburse/generated/train_questions_v1.json
- only artificial answers
Loading augmentation answers from ../../../resources/en/reimburse/generated/train_answers.json
===== Dataset Statistics =====
- files:  en/reimburse/train_graph.json en/reimburse/train_answers.json
- synonyms: True
- depth: 20  - degree: 13
- answers: 803
- questions: 800
- loaded original data: False
- loaded generated data: True
- question limit: 0  - maximum loaded:  10
- answer limit: 0  - maximum loaded:  11
- Loading questions from  ../../../resources/en/reimburse/generated/train_questions_v1_ling.json
- only artificial answers
Loading augmentation answers from ../../../resources/en/reimburse/generated/train_answers.json
===== Dataset Statistics =====
- files:  en/reimburse/train_graph.json en/reimburse/train_answers.json
- synonyms: True
- depth: 20  - degree: 13
- answers: 803
- questions: 3681
- loaded original data: False
- loaded generated data: True
- questi

In [31]:
from data.dataset import NodeType, Question
import time

set_seed(42)

system = """You are a truthful assistant deciding if a given question can be answered only using the presented fact, without any additional external knowledge.
Only reply with "yes" or "no"."""

def user(question_text: str, node_text: str) -> str:
    return f'Can the question "{question_text}" be answered without any additional external knowledge and only using the fact: "{node_text}"'

TEMPERATURE = 0.7
MAX_NEW_TOKENS = 1024
generated_data = {}

for node in data_generated_v1.nodes_by_type[NodeType.INFO]:
    for question in node.questions:
        prompt = generate_prompt(system=system, user=user(question.text, node.text)).strip()
        gen = generate_output(prompt=prompt, temperature=TEMPERATURE, max_new_tokens=MAX_NEW_TOKENS)
        cleaned = parse_output(prompt, gen)
        print(cleaned)
        break
    break

Yes


In [32]:
from data.dataset import NodeType, Question
import time

set_seed(42)

system = """You are a truthful assistant deciding if a given question can be answered only using the presented fact, without any additional external knowledge.
Only reply with "yes" or "no"."""

def user(question_text: str, node_text: str) -> str:
    return f'Can the question "{question_text}" be answered without any additional external knowledge and only using the fact: "{node_text}"'

TEMPERATURE = 0.7
MAX_NEW_TOKENS = 1024
generated_data = {}
responses = set()

for node in tqdm(data_generated_v1.nodes_by_type[NodeType.INFO]):
    for question in node.questions:
        prompt = generate_prompt(system=system, user=user(question.text, node.text)).strip()
        gen = generate_output(prompt=prompt, temperature=TEMPERATURE, max_new_tokens=MAX_NEW_TOKENS)
        cleaned = parse_output(prompt, gen)
        generated_data[question.key] = cleaned
        responses.add(cleaned)

  4%|▍         | 3/80 [10:35<4:31:36, 211.65s/it]

In [ ]:
# TODO save data!

# import json

# cleaned_data = {}
# for key in generated_data:
#     node = human_data_train.nodes_by_key[generated_data[key]['dialog_node_key']]
#     cleaned_data[key] = generated_data[key]
#     for i in range (1, NUM_QUESTIONS+1):
#         cleaned_data[key]['text'] = cleaned_data[key]['text'].replace(f"{i}.", "").strip()
#     cleaned_data[key]["node_text"] = node.text
#     cleaned_data[key]["node_type"] = node.node_type.value

# with open("TODO/train_questions_v1.json", "w") as f:
    # json.dump(cleaned_data, f)